Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos o valor zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



In [1]:
import random

from funcoes import computa_mochila
from funcoes import funcao_objetivo_pop_mochila
from funcoes import populacao_cb as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao

## Códigos e discussão



In [19]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 300
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
LIMITE_DE_PESO = 25
OBJETOS = {
    # dicionário baseado no que vocês enviaram na aula de Lógica

    "Pokebola shine": {
        "peso": 11,
        "valor": 25000,
    },
    "Papaleguas de uma perna feita com grafite": {
        "peso": 3,
        "valor": 6500,
    },
    "Pele": {
        "peso": 0,
        "valor": 1000,
    },
    "Pc Gamer Mediano": {
        "peso": 10,
        "valor": 15000,
    },
    "Picareta do Fortnite": {
        "peso": 3.2,
        "valor": 4400,
    },
    "bola de fogo": {
        "peso": 7.5,
        "valor": 8000,
    },
    "ilum": {
        "peso": 16,
        "valor": 21000,
         },
    "Rhuan e Alê": {
        "peso": 4,
        "valor": 2000,
    },
    "suco de laranja": {
        "peso": 0.02,
        "valor": 500,
    },
    "papaguaio cruzeirense": {
        "peso": 7,
        "valor": 17000,
    },
}
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys()))

In [20]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(
        populacao, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES
    )

In [39]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf")
melhor_individuo_ja_visto = [0] * NUM_OBJETOS

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO:
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")
        
# reportando o melhor individuo encontrado
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
)
print(
    f"Com isso, sua mochila terá o valor de {valor_total} dinheiros "
    f"e peso de {peso_total} unidades de massa."
)

Maior valor: 39500 | Peso: 21.5
Maior valor: 49000 | Peso: 21.02
Maior valor: 49500 | Peso: 21
Maior valor: 50000 | Peso: 21.02
Maior valor: 50500 | Peso: 25
Maior valor: 53400 | Peso: 24.22
Maior valor: 53900 | Peso: 24.2
Maior valor: 54400 | Peso: 24.22

Você deve pegar os seguintes itens:
+ Papaleguas de uma perna feita com grafite
+ Pele
+ Picareta do Fortnite
+ Pokebola shine
+ papaguaio cruzeirense
+ suco de laranja

Com isso, sua mochila terá o valor de 54400 dinheiros e peso de 24.22 unidades de massa.


<p style='text-align: justify'> Este problema, o da mochila, para resolvê-lo foi necessário criar duas funções. Dentre elas, uma que transcreve a função por partes para o formato de código, assim, caso o peso seja superior ao limite estabelecido para a bolsa, uma penalização será realizada, assim o novo valor sendo extremamente baixo, mas não zero. Penalização essa que é efetiva pois o problema é de maximização, logo, valores baixos não serão escolhidos ou estarão no fundo da lista de escolhas. Ambas as funções foram importadas na primeira células e estão bem descritas em "funcoes.py". </p>

<p style='text-align: justify'> Sobre a segunda célula, além das constantes já usadas nos demais exercícios de algoritmos genéticos, sobre o problema, foi necessário determinar a capacidade máxima de peso que a mochila suporta e principalmente quais são esses itens, tanto nome, como peso e valor. Portanto, tentando maximizar esse valor por meio do número de gerações e não extrapolando o peso, visando a penalização. </p>

## Conclusão



<p style='text-align: justify'> O problema da mochila onde há uma limitação de peso, foi resolvido com sucesso. Desse modo, duas principais funções foram criadas, estas que representam o problema matemático em forma de código e aplicam um limite ao máximo de peso suportado pela mochila, sendo condizente ao proposto. Ademais, como o problema visa uma maximização e visando os itens que foram determinados, junto com seus respectivos pesos e valores, o algoritmo tenta alcançar o valor máximo e não extrapolar o peso para evitar a penalização. Contudo o algoritmo é probabilístico, portanto, nem sempre retornara o maior valor e a melhor solução possível, dessa maneira, o número de gerações determina quantas serão as tentativas que o código tentara. Por fim, com essa constante possuindo valor igual a 300, os resultados mesmo que nem sempre o melhor, tem sido satisfatório e próximo do melhor. </p>

## Playground

